# Steigende Temperaturen in Berlin Tegel

Die in diesem Notebook genutzten Daten stammen vom [Deutschen Wetterdienst (DWD)](https://www.dwd.de/)
und werden über das [Climate Data Center (CDC)](https://www.dwd.de/DE/klimaumwelt/cdc/cdc_node.html) zur Verfügung gestellt. 
Konkret werden die
[historischen](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/tageswerte_KL_00430_19630101_20181231_hist.zip)
und
[aktuellen](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/tageswerte_KL_00430_akt.zip)
täglichen Wetterdaten der DWD Messstation 430 in Berlin Tegel seit 1.1.1963 verwendet.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import shapiro
import seaborn as sns

sns.set(rc={'figure.figsize':(20,10)})

data_files = [
    '../data/historisch/produkt_klima_tag_19630101_20181231_00430.txt',
    # '../data/aktuell/produkt_klima_tag_20180215_20190818_00430.txt'
]
df = pd.concat((pd.read_csv(f, 
                            sep=';',
                            header=None,
                            skiprows=1,
                            names=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'FX', 'FM', 'QN_4', 'RSK', 'RSKF', 'SDK', 'SHK_TAG', 'NM', 'VPM', 'PM', 'TMK', 'UPM', 'TXK', 'TNK', 'TGK', 'eor'],
                            usecols=['MESS_DATUM', 'RSK', 'TMK'],
                            na_values=['-999'],
                            parse_dates=['MESS_DATUM'],
                            ) for f in data_files), ignore_index=True)
df.index = df['MESS_DATUM']
df = df.drop(columns=['MESS_DATUM'])

In [ ]:
def add_source(_ax):
    """Hilfsfunktion um Quellenangabe einem Plot hinzuzufügen"""
    _ax.text(1, 1, 'Quelle: Deutscher Wetterdienst (DWD)', ha='right', va='top', transform=_ax.transAxes)

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

Man erkennt an der Standardabweichung im Vergleich zu den Mittelwerten,
dass die Tageswert für Temperatur und Niederschlag wenig hilfreich sein,
da die Messwerte stark saisonal sind (Jahreszeiten).

Dies lässt sich für die Temperaturen auch nochmals in einem Boxplot visualisieren. 

In [ ]:
add_source(sns.boxplot(x=df['TMK']))

Im Fall der Tagestemperaturwerte (TMK) ist es daher sinnvoller, einen jährlichen Durchschnitt zu bilden.

In [ ]:
temp_df = df[['TMK']].resample('Y').mean()
temp_df.head(5)

In [ ]:
temp_df.describe()

In [ ]:
temp_df.info()

In [ ]:
add_source(sns.boxplot(x=temp_df))

In [ ]:
add_source(sns.distplot(temp_df))


Auch wenn die durchschnittliche Jahrestemperatur stark schwankt, sind doch steigende Temperaturen erkennbar.

In [ ]:
temp = sns.lineplot(data=temp_df, legend=False)
temp.set_xlabel('Jahr', fontsize=20)
temp.set_ylabel('durchschn. Jahrestemperatur', fontsize=20)
add_source(temp)

x = np.arange(temp_df['TMK'].size)
fit = np.polyfit(x, temp_df['TMK'], 1)
fit_fn = np.poly1d(fit)

temp.plot(temp_df.index, fit_fn(x), 'k-')

In [ ]:
stat, p = shapiro(temp_df['TMK'])
alpha = 0.05
if p > alpha:
    print('Die jährliche Durschnittstemperatur ist wahrscheinlich normalverteilt.')
else:
    print('Die jährliche Durchschnittstemperatur ist nicht normalverteilt.')

Bei den Regenwerten (RSK) ist die jährliche Regenmenge interessanter als die täglichen Einzelwerte.

In [ ]:
regen_df = df[['RSK']].resample('Y').sum()
regen_df.head(5)

In [ ]:
regen_df.describe()

In [ ]:
regen_df.info()

Das folgende Histogramm deutet daraufhin, dass die jährliche Regenmenge normalverteilt ist.

In [ ]:
add_source(sns.distplot(regen_df))

Auch in folgendem Plot kann ich keine steigende oder sinkende Regenmenge abgelesen.

In [ ]:
regen = sns.lineplot(data=regen_df, legend=False)
regen.set_xlabel('Jahr')
regen.set_ylabel('jährliche Regenmenge in mm')
add_source(regen)

x = np.arange(regen_df['RSK'].size)
fit = np.polyfit(x, regen_df['RSK'], 1)
fit_fn = np.poly1d(fit)
regen.plot(regen_df.index, fit_fn(x), 'k-')

Ich nutze den Shapiro-Wilk Test, um die jährliche Regenmenge auf Normalverteilung zu testen.

In [ ]:
stat, p = shapiro(regen_df['RSK'])
alpha = 0.05
if p > alpha:
    print('Die jährliche Regenmenge ist wahrscheinlich normalverteilt.')
else:
    print('Die jährliche Regenmenge ist nicht normalverteilt.')